In [7]:
import os

In [8]:
os.cpu_count()

16

In [6]:
!pwd

/home/ec2-user/SageMaker


In [40]:
import torch

In [10]:
torch.__version__

'2.1.1+cu121'

In [23]:
!sudo lfs hsm_restore /fsx/data/val/eschscholzia_minutiflora/e7b5513a5ad8dcb04fb12d878a1e716e.jpg

In [3]:
import concurrent.futures
from tqdm import tqdm
import subprocess
from pathlib import Path
import os

def run_command(file):
    cmd = f"sudo lfs hsm_restore {file}"
    result = subprocess.run(cmd, shell=True)

    return True

def execute():
    dirs = sorted(list(Path('/fsx/data/train').iterdir()))
    files = []
    for i in tqdm(range(1000, 5000)):

        files.extend(list(Path(dirs[i]).glob('*.jpg')))
        

        with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
            list(executor.map(run_command, files))

In [4]:
execute()

  0%|          | 36/22986 [02:57<31:27:32,  4.93s/it]


KeyboardInterrupt: 

In [11]:
from time import perf_counter
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
from transformers import (
    AutoConfig,
    AutoImageProcessor,
)
model_name_or_path = 'microsoft/swin-base-patch4-window7-224-in22k'
train_dir = '/fsx/data/val'

image_processor = AutoImageProcessor.from_pretrained(
    model_name_or_path
)

normalize = Normalize(
    mean=image_processor.image_mean, std=image_processor.image_std
)

_train_transforms = Compose(
    [
        RandomResizedCrop(224),
        ToTensor(),
        normalize,
    ]
)
time_start = perf_counter()
train_ds = ImageFolder(train_dir, transform=_train_transforms)
print(f"Executed: {perf_counter()-time_start}")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Executed: 73.50190370199925


In [18]:
import os
# persistent_workers=True
loader = DataLoader(train_ds, batch_size=32, num_workers=16, persistent_workers=True)
it = iter(loader)

for _ in range(20):
    time_start = perf_counter()
    r = next(it)
    print(f"Executed: {(perf_counter()-time_start)*1000:.2f}ms per iter")


Executed: 5.29ms per iter
Executed: 9.25ms per iter
Executed: 1.90ms per iter
Executed: 1.95ms per iter
Executed: 1.87ms per iter
Executed: 2.33ms per iter
Executed: 2.39ms per iter
Executed: 2.57ms per iter
Executed: 2.40ms per iter
Executed: 2.16ms per iter
Executed: 2.79ms per iter
Executed: 2.29ms per iter
Executed: 2.29ms per iter
Executed: 2.85ms per iter
Executed: 2.81ms per iter
Executed: 4.54ms per iter
Executed: 16.80ms per iter
Executed: 6.60ms per iter
Executed: 2.86ms per iter
Executed: 2.68ms per iter
